In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk, os, re, string, collections
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from  imageio import imread
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import multidict as multidict #pip install multidict
import itertools
#import csvtomd
import re
from collections import Counter
#%matplotlib inline
import spacy
from sklearn.preprocessing import MultiLabelBinarizer, Binarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 1. Limpieza y conversión de datos

In [2]:
dtype_test = {"id": np.int32, "keyword": "category"}
test = pd.read_csv("original_data/test.csv", dtype = dtype_test)

In [3]:
dtype_train = {"id": np.int32, "keyword": "category", "target" : int}
train = pd.read_csv("original_data/train.csv", dtype = dtype_train, encoding='UTF_8')

In [4]:
target_train = train.target

## 1.1 Keywords

In [5]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatize_text(text):
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer  = nltk.stem.WordNetLemmatizer()
    if text == '' or pd.isnull(text):
        return text
    else:
        wordList = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
        s = ' '
        return s.join(wordList)
    
def lemmatize_text_v2(text):
    if text == '' or pd.isnull(text):
        return text
    else:
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

def clean_keyword_field(df):
    df["keyword"] = df["keyword"].str.replace('%20',' ')
    df_sin_keywords_nulos = df.dropna(subset = ['keyword'])
    keywords = np.unique(df_sin_keywords_nulos.keyword)
    for keyword in keywords:
        df.loc[(df['text'].str.contains(keyword, case=False) & df['keyword'].isnull()), 'keyword'] = keyword
    #df['keyword'] = df['keyword'].apply(lemmatize_text_v2)
    return df

## 1.2 Text

### Strict (for tags)

In [6]:
def getStopwordsList():
    fileNamesList = ["texts/99webtools.txt", "texts/atire_ncbi.txt", "texts/atire_puurula.txt", "texts/azure.txt", 
                 "texts/bbalet.txt", 
                 "texts/bow_short.txt", "texts/choi_2000naacl.txt", "texts/cook1988_function_words.txt", 
                 "texts/corenlp_acronym.txt", 
                 "texts/corenlp_hardcoded.txt", "texts/corenlp_stopwords.txt", "texts/datasciencedojo.txt", 
                 "texts/deeplearning4j.txt", 
                 "texts/dkpro.txt", "texts/mongodb.txt", "texts/galago_inquery.txt", "texts/gate_keyphrase.txt", 
                 "texts/gensim.txt", 
                 "texts/glasgow_stop_words.txt", "texts/indri.txt", "texts/kevinbouge.txt", "texts/lexisnexis.txt",
                 "texts/lingpipe.txt", 
                 "texts/mallet.txt", "texts/mysql_innodb.txt", "texts/mysql_myisam.txt", "texts/galago_rmstop.txt", 
                 "texts/atire_ncbi.txt", 
                 "texts/galago_rmstop.txt", "texts/nltk.txt", "texts/okapiframework.txt", "texts/okapi_cacm_expanded.txt", 
                 "texts/onix.txt", 
                 "texts/ovid.txt", "texts/postgresql.txt", "texts/pubmed.txt", "texts/quanteda.txt", "texts/r_tm.txt", 
                 "texts/ranksnl_large.txt", 
                 "texts/reuters_wos.txt", "texts/rouge_155.txt", "texts/scikitlearn.txt", "texts/smart.txt", 
                 "texts/snowball_expanded.txt", 
                 "texts/spacy.txt", "texts/spark_mllib.txt", "texts/sphinx_mirasvit.txt", "texts/t101_minimal.txt", 
                 "texts/taporware.txt", 
                 "texts/terrier.txt", "texts/tonybsk_1.txt", "texts/tonybsk_6.txt", "texts/voyant_taporware.txt", 
                 "texts/weka.txt", 
                 "texts/xapian.txt", "texts/xpo6.txt", "texts/zettair.txt"]
    stopwordsList = []
    for fileName in fileNamesList:
        file = open(fileName, "r", encoding="utf8")
        for line in file:
            stripped_line = line. strip()
            line_list = stripped_line
            if line_list not in stopwordsList:
                stopwordsList.append(line_list)
        file.close()
    return stopwordsList

stopwordsList = getStopwordsList()

print(len(stopwordsList))

2253


In [7]:
def clean_text_strict(text):
    text = text.lower()
    text = text.replace('\d+', '')
    invalid_chars = ['#','|','@','!','?','-','_','[',']','%','&',':','.',',',"''",'/','https','(','//t',')','http',
                 ';','\'']
    for char in invalid_chars:
        if char in text:
            text = text.replace(char,' ')
    #removes url and tags
    text = re.sub(r"(?:\@|https?\://)\S+", " ",  text)
    return text

### Non strict (for special characters, hashtags, etc)

In [8]:
def clean_text_non_strict(text): 
    tw = " "
    words = text.split(' ')
    for word in words:
        word = ''.join(filter(lambda x: x in set(string.printable), word))
        word = word.replace("\n"," ")
        word = word.replace('û',"")
        word = word.replace('Û',"")
        #word = word.replace('_','')
        #word = word.replace("\"",'')
        #word = word.strip('.')
        #word = word.strip(',')
        #word = word.strip(':')
        tw += word + " "
    return tw

## 1.3 Location

In [9]:
#FUNCIONES AUXILIARES: Limpieza de tweet

# Función que saca los links

def quitar_link_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if (("http" not in w) and ("https" not in w)):
            res.append(w)
    return (" ").join(res)

# Función que saca las menciones

def quitar_mencion_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if("@" not in w):
            res.append(w)
    return (" ").join(res)

def quitar_simbolo(tweet, simbolo):
    text = ""
    for c in tweet:
        if(c!=simbolo):
            text+=c
    return text

def agregar_espacio_ente_comas(tweet):
    
    tweer_sin_coma = (" ").join(tweet.split(","))

    return (" ").join(tweer_sin_coma.split("."))

def capitalize_each_word(string):
    
    text = ""
    for w in string.split(" "):
        if(len(w)>3):
            text += w.capitalize()+" "
    return text

def limpiar_tweet_location(tweet):
    res = tweet
    func = [quitar_mencion_twitter, quitar_link_twitter, agregar_espacio_ente_comas, lambda x: quitar_simbolo(x, "#"), capitalize_each_word]
    for f in func:
        res = f(res)
    return res

def to_string(a_list, delimeter):
    res = ""
    
    for e in sorted(filter(None, a_list)):
        res += str(e)+delimeter
    
    if(len(res)==0): return None
    return res[: -len(delimeter)]

In [10]:
# FUNCIONES AUXILIARES: Analisis geografico

from geotext import GeoText #pip install https://github.com/elyase/geotext/archive/master.zip
import pycountry
import geonamescache # pip install geonamescache

gc = geonamescache.GeonamesCache()

def get_country_pycountry(text):
    for country in pycountry.countries:
        # Handle both the cases(Uppercase/Lowercase)
        if str(country.name).lower() in str(text).lower():
            return GeoText(country.name).country_mentions

def get_country_geotext(text):
    return GeoText(text).country_mentions

def get_country(text):
    set1 = set(get_country_geotext(text))
    countries = get_country_pycountry(text)
    if(not countries):
        return list(set1)
    return list(set1.union(set(countries)))

def check_if_US_state(text):
    stateList = []
    states = gc.get_us_states_by_names()
    for state in states:
        if (state in text):
            stateList.append(state)
    return list(set(stateList))

def check_if_US_county(text):
    countyList = []
    stateList = []
    counties = gc.get_us_counties()
    for county in counties:
        name = county.get('name')
        if name in text:
            countyList.append(name)
            stateList.append(county.get('state'))
    return list(set(countyList))

def check_if_continent(text):
    continentList = []
    continents = gc.get_continents()
    for continent,info in continents.items():
        if (info.get('asciiName') in text):
            continentList.append(continent)
    return continentList

def get_US_state(county):
    stateList = []
    counties = gc.get_us_counties()
    if (pd.isnull(county)):
        return county
    for c in counties:
        name = c.get('name')
        if name in county:
            stateList.append(c.get('state'))
    return list(set(stateList))

def get_continent(country):
    continentList = []
    countries = gc.get_countries()
    if (pd.isnull(country)):
        return continentList
    country = country.split(',')
    for c in country:
        info = countries.get(c)
        continentList.append(info.get('continentcode'))
    return list(set(continentList))

### Location in text

In [11]:
# Creo dataframe y limpio la location

#location_in_text = train[["id", "text"]].copy()
#location_in_text['text'] = location_in_text['text'].apply(lambda x: limpiar_tweet_location(x))

In [12]:
#Creo columnas con informacion
#location_in_text["cities"] = location_in_text["text"].apply(lambda x: GeoText(str(x)).cities)
#location_in_text["cities_str"] = location_in_text["cities"].apply(lambda x: to_string(x,","))

#location_in_text["country"] = location_in_text["text"].apply(lambda x: get_country(x))

#location_in_text["counties"] = location_in_text["text"].apply(lambda x: check_if_US_county(str(x)))
#location_in_text["counties_str"] = location_in_text["counties"].apply(lambda x: to_string(x,","))

#location_in_text["states"] = location_in_text["text"].apply(lambda x: check_if_US_state(str(x)))
#location_in_text.loc[(location_in_text['states'].str.len() == 0), 'states'] = location_in_text.counties.apply(lambda x: get_US_state(x))

#location_in_text["country_str"] = location_in_text["country"].apply(lambda x: to_string(x,","))
#location_in_text["states_str"] = location_in_text["states"].apply(lambda x: to_string(x,","))
#location_in_text.loc[~pd.isnull(location_in_text['counties_str']), 'country_str'] = 'US'
#location_in_text.loc[~pd.isnull(location_in_text['states_str']), 'country_str'] = 'US'

#location_in_text["continent"] = location_in_text["text"].apply(lambda x: check_if_continent(str(x)))
#location_in_text.loc[location_in_text['continent'].str.len() == 0, 'continent'] = location_in_text.country_str.apply(lambda x: get_continent(x))

#location_in_text["continent_str"] = location_in_text["continent"].apply(lambda x: to_string(x,","))

#location_in_text["has_country"] = np.where(pd.isnull(location_in_text['country_str']), False, True)

#location_in_text.head()

In [13]:
# Selecciono las columnas más útiles para crear features

#location_in_text = location_in_text[["id","counties_str","cities_str","states_str","country_str","continent_str","has_country"]]
#location_in_text.columns = ["id", "has_location", "cities", "country"]
#location_in_text.head(5)

In [14]:
#location_in_text.head()

### Location label info

In [15]:
# Creo dataframe y limpio la location

#location = train[["id", "location","target"]].copy().fillna("")
#location["location"] = location["location"].apply(lambda x: limpiar_tweet_location(x))

In [16]:
#Creo columnas con informacion

#location["has_location"] = location["location"]!=""

#location["cities"] = location["location"].apply(lambda x: GeoText(str(x)).cities)
#location["cities_str"] = location["cities"].apply(lambda x: to_string(x,","))

#location["country"] = location["location"].apply(lambda x: get_country(x))
#location["country_str"] = location["country"].apply(lambda x: to_string(x,","))

#location["cities"] = location["location"].apply(lambda x: GeoText(str(x)).cities)
#location["cities_str"] = location["cities"].apply(lambda x: to_string(x,","))

#location["country"] = location["location"].apply(lambda x: get_country(x))

#location["counties"] = location["location"].apply(lambda x: check_if_US_county(str(x)))
#location["counties_str"] = location["counties"].apply(lambda x: to_string(x,","))

#location["states"] = location["location"].apply(lambda x: check_if_US_state(str(x)))
#location.loc[(location['states'].str.len() == 0), 'states'] = location.counties.apply(lambda x: get_US_state(x))

#location["country_str"] = location["country"].apply(lambda x: to_string(x,","))
#location["states_str"] = location["states"].apply(lambda x: to_string(x,","))
#location.loc[~pd.isnull(location['counties_str']), 'country_str'] = 'US'
#location.loc[~pd.isnull(location['states_str']), 'country_str'] = 'US'

#location["continent"] = location["location"].apply(lambda x: check_if_continent(str(x)))
#location.loc[location['continent'].str.len() == 0, 'continent'] = location.country_str.apply(lambda x: get_continent(x))

#location["continent_str"] = location["continent"].apply(lambda x: to_string(x,","))

#location["has_country"] = np.where(pd.isnull(location['country_str']), False, True)

#location[location['has_location']==True].head(50)

In [17]:
#location_aux = location[["country_str", "cities_str", "states_str", "continent_str", "counties_str"]].isnull()[520:525]
#location["city_country_not_found"] = location_aux["country_str"] & location_aux["cities_str"] & location_aux["counties_str"] & location_aux["continent_str"] & location_aux["states_str"]

In [18]:
# Selecciono las columnas más útiles para crear features

#location_info = location[["id", "has_location", "cities_str", "country_str", "city_country_not_found", "counties_str", "states_str", "continent_str"]]
#location_info.columns = ["id", "has_location", "cities", "country", "city_country_not_found", "county", "state", "continent"]

In [19]:
#location_info.head()

## 1.4 Links

In [20]:
links_train = pd.read_csv("new_data/links_train.csv", encoding='UTF_8').set_index("id")
links_test = pd.read_csv("new_data/links_test.csv", encoding='UTF_8').set_index("id")

In [21]:
import ast

links_train['http']= links_train['http'].apply(lambda x: ast.literal_eval(x))
links_test['http']= links_test['http'].apply(lambda x: ast.literal_eval(x))

In [22]:
#Funciones auxiliares

def get_dominio(links):
    res= []
    for l in links:
        if(l==None):
            res.append(None)
        else:
            dom = l.split("/")
            res.append(dom[2])
    return res

def type_http(links):
    res = []
    for l in links:
        if(l==None):
            res.append(None)
        else:
            dom = l.split(":")
            res.append(dom[0])
    return res

def count_None(a_list):
    count = 0
    for l in a_list:
        if(l==None):
            count+=1
    return count

def link_encription_generalize(link_encript):
    
    has_http = False
    has_https = False
    
    for type_http in link_encript:
        if type_http==None: 
            continue
        elif type_http== "https":
            has_https = True
        else:
            has_http = True
    
    if(has_http and has_https):
        return "both"
    elif(has_http):
        return "http"
    elif(has_https):
        return "https"
    else:
        return None
    
def to_string(a_list, delimeter):
    res = ""
    
    for e in sorted(filter(None, a_list)):
        res += str(e)+delimeter
    
    if(len(res)==0): return None
    return res[: -len(delimeter)]

## 1.5 Tags

### Limpieza de los campos para obtener tags

In [23]:
# FUNCIONES AUXILIARES

def prettify(word_list):
    wl = []
    for word in word_list:
        word = re.sub(r'[^A-Za-z]', " ",  word)
        word = re.sub(r'\b\w\b',' ', word)
        word = re.sub(r'\s+', ' ', word)
        wl.append(word.strip())
    return wl

def lemmatize_text2(word_list):
    text = ' '
    text = text.join(word_list)
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

def getText(word_list):
    text = ' '
    text = text.join(word_list)
    return text

def word_finder(x):
    x = set(x)
    return list(unique_frequent_words.intersection(x))

In [24]:
# Creo el dataframe
#df_tags = train[['id','text','target']].copy(deep=True)

# Limpio el dataframe
#df_tags['new'] = df_tags["text"].apply(lambda x: clean_text_strict(x))

In [25]:
#  Eliminacion de terminos, caracteres y stopwords innecesarios para generar los tags
#convierte de list a string

#df_tags['new'] = df_tags['new'].apply(lambda x: [item for item in x.split() if item not in stopwordsList])
#df_tags.new = df_tags.new.apply(prettify)

#df_tags.head()

### Tokenize

In [26]:
#w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
#lemmatizer = nltk.stem.WordNetLemmatizer()

#df_tags['new_tokenized'] = df_tags.new.apply(lemmatize_text2)
#df_tags.new_tokenized = df_tags.new_tokenized.apply(prettify)

#df_tags.head()

### Analisis de ocurrencias

In [27]:


#metodo para analizar ocurrencias y cantidades
#a = df_tags['new_tokenized'].apply(getText).str.lower().str.cat(sep=' ')

#words = nltk.tokenize.word_tokenize(a)
#word_dist = nltk.FreqDist(words)

#this_col = pd.Series(['word','frequency'])

#rslt = pd.DataFrame(word_dist.most_common(400),columns = this_col)

In [28]:
#frequent_words = rslt ['word']
#unique_frequent_words = set(frequent_words)

### Se genera columna con tags a partir de una cantidad de ocurrencias destacadas

# 2. Features

## 2.1 Keywords

In [29]:
train_keywords = train.copy()
train_keywords = clean_keyword_field(train_keywords)

In [30]:
train_keywords_lemmatized = train_keywords.copy()
train_keywords_lemmatized.keyword = train_keywords.keyword.apply(lambda x: lemmatize_text_v2(x))

In [31]:
train_just_links = links_train.reset_index()[['http','id']]
train_keywords_links = train_keywords.merge(train_just_links, on='id', how='left')

In [32]:
test_keywords = test.copy()
test_keywords = clean_keyword_field(test_keywords)

In [33]:
test_keywords_lemmatized = test_keywords.copy()
test_keywords_lemmatized.keyword = test_keywords.keyword.apply(lambda x: lemmatize_text_v2(x))

In [34]:
test_just_links = links_test.reset_index()[['http','id']]
test_keywords_links = test_keywords.merge(test_just_links, on='id', how='left')

In [35]:
def getKeywords(df):
    df_sin_keywords_nulos = df.dropna(subset = ['keyword'])
    return np.unique(df_sin_keywords_nulos.keyword)

keywords_lemmatized_train = getKeywords(train_keywords_lemmatized)
keywords_train = getKeywords(train_keywords)
keywords_lemmatized_test = getKeywords(test_keywords_lemmatized)
keywords_test = getKeywords(test_keywords)

def countKeywords(tweet, train):
    if (train == True):
        keywords = keywords_train
    else:
        keywords = keywords_test
    quantity = 0
    tweet = re.sub('[^a-zA-Z\ ]+', "", tweet).lower()
    for word in tweet.split(' '):
        #word = re.sub('[^a-zA-Z]+', '', word)
        if word in keywords:
            quantity+=1
    return quantity

def getHiddenKeywords(tweet, train):
    if (train == True):
        keywords = keywords_train
    else:
        keywords = keywords_test
    k = []
    tweet = re.sub('[^a-zA-Z\ ]+', "", tweet).lower()
    for keyword in keywords:
        if keyword in tweet:
            k.append(lemmatize_text_v2(keyword))
    return k

In [36]:
def toText(listOfWords):
    s=' '
    return s.join(listOfWords)

def textContainsKeyword(row):
    if row.keyword == '' or pd.isnull(row.keyword):
        return False
    else:
        return row.keyword.lower() in row.text.lower()

def textContainsKeywordAsHashtag(row):
    hashtags = []
    tweet = row.text.lower()
    if row.keyword == '' or pd.isnull(row.keyword):
        return False
    else:
        for word in tweet.split(' '):
            if (len(word)>1) and (word[0] == '#'):
                hashtags.append(word[1:])
        return row.keyword.lower() in hashtags

def linkContainsKeyword(row):
    links   = row.http
    keyword = row.keyword
    if row.keyword == '' or pd.isnull(row.keyword):
        return False
    if (type(links)==list):
        for link in links:
            if link=='' or pd.isnull(link):
                continue
            if keyword in link:
                return True
    else:
        return False
    return False

def keywords_agregar_features_numericos(df, train):
    if (train == True):
        df_keywords_lemmatized = train_keywords_lemmatized
        keywords_lemmatized    = keywords_lemmatized_train
        keywords               = keywords_train
    else:
        df_keywords_lemmatized = test_keywords_lemmatized
        keywords_lemmatized    = keywords_lemmatized_test
        keywords               = keywords_test
    df['text_contains_keyword'] = (df.apply(textContainsKeyword,axis=1)).astype(int)
    df['has_keyword'] = (~df.keyword.isnull()).astype(int)
    df['keywords_quantity'] = df.text.apply(countKeywords, args=(train,))
    df['keywords_mean'] = df.text.apply(countKeywords, args=(train,))/keywords.size
    df['keyword_is_hashtag'] = (df.apply(textContainsKeywordAsHashtag,axis=1)).astype(int)
    df['keyword_frequency'] = df_keywords_lemmatized.groupby('keyword')['id'].transform('count') / keywords_lemmatized.size
#    df['link_contains_keyword'] = (df_keywords_links.apply(linkContainsKeyword,axis=1)).astype(int)
    return df

def keywords_agregar_features_categoricos(df, train):
    df['keywords_in_text'] = df.text.apply(getHiddenKeywords, args=(train,))
    return df

In [37]:
#all_keywords = pd.get_dummies(df_keywords_lemmatized[['id','keyword']],'keyword')
#all_keywords.head()

In [38]:
#df_keywords = df_train.copy()

In [39]:
#df_sin_keywords_nulos = df_keywords.dropna(subset=['keyword'])
#%time df_keywords_not_in_text = df_sin_keywords_nulos[~df_sin_keywords_nulos['text'].str.contains("|".join(df_sin_keywords_nulos['keyword']),case=False)]
#%time df_sin_keywords_nulos[df_sin_keywords_nulos.apply(lambda x: x.keyword in x.text, axis=1)]

In [40]:
keywords_numerical_features = train_keywords.copy()
keywords_numerical_features = keywords_agregar_features_numericos(keywords_numerical_features, True).drop(columns=['location','text','keyword'])
keywords_numerical_features.head()

,id,target,text_contains_keyword,has_keyword,keywords_quantity,keywords_mean,keyword_is_hashtag,keyword_frequency
0,1,1,1,1,1,0.004525,1,0.227027
1,4,1,1,1,1,0.004525,0,0.237838
2,5,1,1,1,1,0.004525,0,0.210811
3,6,1,1,1,1,0.004525,0,0.210811
4,7,1,1,1,1,0.004525,0,0.237838


### Encoding de variables categoricas

In [219]:
keywords_categorical_features = train_keywords.copy() 
keywords_categorical_features = keywords_agregar_features_categoricos(keywords_categorical_features,True).drop(columns=['keyword','location','text'])
keywords_categorical_features.head()

,id,target,keywords_in_text
0,1,1,[earthquake]
1,4,1,"[fire, forest fire]"
2,5,1,[evacuation]
3,6,1,"[evacuation, fire, wildfire]"
4,7,1,"[fire, smoke, wildfire]"


In [42]:
mlb = MultiLabelBinarizer()

keywords_categorical_features = keywords_categorical_features.join(pd.DataFrame(mlb.fit_transform(keywords_categorical_features.pop('keywords_in_text')),
                          columns=mlb.classes_,
                          index=keywords_categorical_features.index).add_prefix('K_'))
keywords_categorical_features

,id,target,K_ablaze,K_accident,K_aftershock,K_airplane accident,K_ambulance,K_annihilate,K_annihilation,K_apocalypse,...,K_volcano,K_war zone,K_weapon,K_whirlwind,K_wild fire,K_wildfire,K_windstorm,K_wound,K_wreck,K_wreckage
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,7,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6,10,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,13,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,14,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,15,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Agrego features a set de test

In [43]:
test_numerical_keywords   = test_keywords.copy()
test_categorical_keywords = test_keywords.copy()
test_numerical_keywords   = keywords_agregar_features_numericos(test_numerical_keywords,False).drop(columns=['location','text','keyword'])
test_categorical_keywords = keywords_agregar_features_categoricos(test_categorical_keywords,False).drop(columns=['location','text','keyword'])
test_categorical_keywords.head()

,id,keywords_in_text
0,0,[crash]
1,2,[earthquake]
2,3,"[fire, forest fire]"
3,9,"[apocalypse, fire, wildfire]"
4,11,[typhoon]


In [44]:
test_categorical_keywords = test_categorical_keywords.join(pd.DataFrame(mlb.transform(test_categorical_keywords.pop('keywords_in_text')),
                          columns=mlb.classes_,
                          index=test_categorical_keywords.index).add_prefix('K_'))
test_categorical_keywords.head()

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['cliff fall'] will be ignored
  .format(sorted(unknown, key=str)))


,id,K_ablaze,K_accident,K_aftershock,K_airplane accident,K_ambulance,K_annihilate,K_annihilation,K_apocalypse,K_armageddon,...,K_volcano,K_war zone,K_weapon,K_whirlwind,K_wild fire,K_wildfire,K_windstorm,K_wound,K_wreck,K_wreckage
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Dataframes finales

In [45]:
#keywords_numerical_features.to_csv('features/train/features_keywords_numerical.csv', encoding='utf-8')
#keywords_categorical_features.to_csv('features/train/features_keywords_in_text_encoded.csv', encoding='utf-8')

In [46]:
#test_numerical_keywords.to_csv('features/test/features_keywords_numerical.csv', encoding='utf-8')
#test_categorical_keywords.to_csv('features/test/features_keywords_in_text_encoded.csv', encoding='utf-8')

In [47]:
#corrmap = df_keywords.corr()
#fig = plt.figure(figsize=(20,10))

#sns.heatmap(corrmap, vmax=.8, square=True)
#plt.show()

## 2.2 Text

In [261]:
#Funciones auxiliares
stopword = stopwords.words('english')

def stop(text):
    return [w for w in text.split() if w in stopword]

def length(text):
    return(np.mean([len(w) for w in text.split()]))

def punct(text):
    return [c for c in text if c in string.punctuation]

def title(text):
    return [w for w in text.split() if w.istitle()]

def upper_list(text):
    return [w for w in text.split() if w.isupper()]

def lower_list(text):
    return [w for w in text.split() if w.islower()]

def count_special_char(text):
    special_char = 0
    text = text.replace(" ", "")
    for i in range(0, len(text)):
        ch = text[i]
        if (ch.isalpha()):  
            continue
        elif (ch.isdigit()):
            continue
        else:
            special_char += 1
    return special_char

def syllables(text):
    count = 0
    vowels = 'aeiouy'
    for word in text:
        word= word.lower()
        if word[0] in vowels:
            count +=1
        for index in range(1,len(word)):
            if word[index] in vowels and word[index-1] not in vowels:
                count +=1
        if word.endswith('e'):
            count -= 1
        if word.endswith('le'):
            count+=1
        if count == 0:
            count +=1
    return count

def quitar_link_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if (("http" not in w) and ("https" not in w)):
            res.append(w)
    return (" ").join(res)

def quitar_mencion_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if("@" not in w):
            res.append(w)
    return (" ").join(res)

def getHashtags(tweet):
    pat = re.compile(r"#(\w+)")
    return pat.findall(tweet)

def getTaggedUsers(tweet):
    pat = re.compile(r"@(\w+)")
    return pat.findall(tweet)

def agregar_features_cantidad_palabras(df_og):
    df_og['text'] = df_og['text'].apply(lambda x: quitar_link_twitter(x))
    df_og['text'] = df_og['text'].apply(lambda x: quitar_mencion_twitter(x))
    palabras_list = [df_og]
    for df in palabras_list:
        # Número de palabras usadas
        df['#palabras'] = df['text'].apply(lambda x: len(x.split()))
        # Número de palabras únicas
        df['#palabras_unicas'] =  df['text'].apply(lambda x: len(set(x.split())))
        # Número de caracteres
        df['#caracteres'] =  df['text'].apply(lambda x: len(x))
        # Número de stopwords
        df['#stopwords'] = df['text'].apply(lambda x: len(stop(x)))
        # Número de caracteres de puntuación
        df['#puntuacion'] = df['text'].apply(lambda x: len(punct(x)))
        # Número de palabras Capitalizadas
        df['#capitalize'] = df['text'].apply(lambda x: len(title(x)))
        # Número de palabras MAYUSCULAS
        df['#mayusculas'] = df['text'].apply(lambda x: len(upper_list(x)))
        # Número de silabas
        df['#silabas'] = df['text'].apply(lambda x: syllables(x))
        # Promedio de longitud del tweet
        df['promedio_len_word'] = df['text'].apply(lambda x: length(x))
        # Número de caracteres especiales
        df['#caracteres_especiales'] = df['text'].apply(lambda x: count_special_char(x))
    return df

def getUniqueHashtagsList(hashtags):
    hashtagList = []
    for hashtag in hashtags:
        for h in hashtag:
            if h not in hashtagList:
                hashtagList.append(h.lower())
    return hashtagList

def getHashtagsList(hashtags):
    hashtagList = []
    for hashtag in hashtags:
        for h in hashtag:
            hashtagList.append(h.lower())
    return hashtagList

def getMostFrequentHashtag(hashtags):
    maxValue = 0
    finalHashtag = ''
    if (len(hashtags) > 0):
        for hashtag in hashtags:
            hashtag = hashtag.lower()
            if (hashtagDict.get(hashtag) > maxValue):
                finalHashtag = hashtag
                maxValue = hashtagDict.get(hashtag)

    return finalHashtag

def getHashtagFrequency(hashtag):
    if hashtag != '':
        return hashtagDict.get(hashtag)/len(hashtagDict)
    else:
        return 0
    
def getUniqueUsersList(users):
    userList = []
    for user in users:
        for u in user:
            if u not in userList:
                userList.append(u.lower())
    return userList

def getUsersList(users):
    userList = []
    for user in users:
        for u in user:
            userList.append(u.lower())
    return userList

def hasRelevantTaggedUser(users):
    minValue = 5
    #finalUser = ''
    if (len(users) > 0):
        for user in users:
            user = user.lower()
            if (userDict.get(user) > minValue):
                return True
    return False

def text_agregar_features_numericos(df):
    df = agregar_features_cantidad_palabras(df.copy())
    df['#palabras_binned'] = pd.qcut(df['#palabras'], q=4, labels=False)
    df['#palabras_unicas_binned'] = pd.qcut(df['#palabras_unicas'], q=4, labels=False)
    df['#caracteres_binned'] = pd.qcut(df['#caracteres'], q=5, labels=False)
    df['#stopwords_binned']  = pd.qcut(df['#stopwords'], q=3, labels=False)
    df['#puntuacion_binned'] = pd.qcut(df['#puntuacion'], q=3, labels=False)
    df['#capitalize_binned'] = pd.qcut(df['#capitalize'], q=3, labels=False)
    df['#mayusculas_binned'] = pd.qcut(df['#mayusculas'], q=2, labels=False, duplicates='drop')
    df['#silabas_binned']    = pd.qcut(df['#silabas'], q=4, labels=False)
    df['#caracteres_especiales_binned'] = pd.qcut(df['#caracteres_especiales'], q=3, labels=False)
    return df

def hashtags_agregar_features_numericos(df):
    df['hashtags']    = df.text.apply(lambda x: getHashtags(x))
    df['#hashtags']   = df.hashtags.apply(lambda x: len(x))
    df['has_hashtag'] = (df['#hashtags']>0).astype(int)
#    hashtags    = getUniqueHashtagsList(df.hashtags)
#    hashtagDict = Counter(getHashtagsList(df.hashtags))
#    df['relevant_hashtag']      = df.hashtags.apply(lambda x: getMostFrequentHashtag(x))
#    df['freq_relevant_hashtag'] = df.relevant_hashtag.apply(lambda x:getHashtagFrequency(x))
    df = df.drop(columns=['hashtags'])
    return df

def hashtags_agregar_features_categoricos(df):
    df['hashtags'] = df.text.apply(lambda x: getHashtags(x))
    #df_text['relevant_hashtag'] = df_text.hashtags.apply(lambda x: getMostFrequentHashtag(x))
    return df

def tagged_users_agregar_features_numericos(df):
    df['tagged_users'] = df.text.apply(lambda x: getTaggedUsers(x))
    df['#tagged_users'] = df.tagged_users.apply(lambda x: len(x))
#    users = getUniqueUsersList(df.tagged_users)
#    userDict = Counter(getUsersList(df.tagged_users))
#    df['has_tagged_user'] = (df['#tagged_users']>0).astype(int)
#    df['has_relevant_tagged_user'] = (df.tagged_users.apply(lambda x:hasRelevantTaggedUser(x))).astype(int)
    df = df.drop(columns=['tagged_users'])
    return df

def tagged_users_agregar_features_categoricos(df):
    df['tagged_users'] = df.text.apply(lambda x: getTaggedUsers(x))
    return df

In [49]:
df_text = train.copy()
df_text = df_text.drop(columns=['keyword','location'])
df_text.text = df_text.text.apply(clean_text_non_strict)

In [50]:
df_test_text = test.copy()
df_test_text = df_test_text.drop(columns=['keyword','location'])
df_test_text.text = df_test_text.text.apply(clean_text_non_strict)

#### Features NUMERICAL TEXT

#### Features TEXT

In [264]:
text_numerical_features = df_text.copy()
text_numerical_features = text_agregar_features_numericos(text_numerical_features).drop(columns=['text'])
text_numerical_features.head()

,id,target,#palabras,#palabras_unicas,#caracteres,#stopwords,#puntuacion,#capitalize,#mayusculas,#silabas,...,#caracteres_especiales,#palabras_binned,#palabras_unicas_binned,#caracteres_binned,#stopwords_binned,#puntuacion_binned,#capitalize_binned,#mayusculas_binned,#silabas_binned,#caracteres_especiales_binned
0,1,1,13,13,71,5,1,5,1,19,...,1,1,1,1,1,0,2,0,1,0
1,4,1,7,7,40,0,1,5,0,10,...,1,0,0,0,0,0,2,0,0,0
2,5,1,22,20,135,9,3,2,0,27,...,3,3,3,4,2,1,1,0,3,1
3,6,1,8,8,67,1,2,1,0,17,...,2,0,0,1,0,0,0,0,1,0
4,7,1,16,15,90,6,2,3,0,24,...,2,2,2,2,2,0,1,0,2,0


In [52]:
test_text_numerical_features = df_test_text.copy()
test_text_numerical_features = text_agregar_features_numericos(test_text_numerical_features).drop(columns=['text'])

#### Features ARROBAS

In [53]:
tagged_users_numerical_features = df_text.copy()
tagged_users_numerical_features = tagged_users_agregar_features_numericos(tagged_users_numerical_features).drop(columns=['text'])
tagged_users_categorical_features = df_text.copy()
tagged_users_categorical_features = tagged_users_agregar_features_categoricos(tagged_users_categorical_features).drop(columns=['text'])

In [54]:
test_tagged_users_numerical_features = df_test_text.copy()
test_tagged_users_numerical_features = tagged_users_agregar_features_numericos(test_tagged_users_numerical_features).drop(columns=['text'])
test_tagged_users_categorical_features = df_test_text.copy()
test_tagged_users_categorical_features = tagged_users_agregar_features_categoricos(test_tagged_users_categorical_features).drop(columns=['text'])
test_tagged_users_numerical_features

,id,#tagged_users
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
5,12,0
6,21,0
7,22,0
8,27,0
9,29,0


TF-IDF - ARROBAS

In [55]:
tagged_users_categorical_features['tagged_users'] = tagged_users_categorical_features['tagged_users'].apply(lambda x:toText(x))
tagged_index = tagged_users_categorical_features['tagged_users'].fillna('').index
tagged_id = tagged_users_categorical_features['id'].fillna('')
corpus = tagged_users_categorical_features['tagged_users'].fillna('')

# Without Smoothing:
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_tagged=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_tagged.set_index(tagged_index)
df_tf_idf_tagged['id'] = tagged_id

df_tf_idf_tagged.head()

,1061thetwister,115film,15moferadio,1acd4900c1424d1,1rockstar62,20skyhawkmm20,233liveonline,2for1lapdances,4tiles,4tp_kotc,...,zaynmalik,zenande_mcfen,zeno001,zepp1978,zhenghxn,zojadelin,zomatoaus,zourryart,zxathetis,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [56]:
test_tagged_users_categorical_features['tagged_users'] = test_tagged_users_categorical_features['tagged_users'].apply(lambda x:toText(x))
test_tagged_index = test_tagged_users_categorical_features['tagged_users'].fillna('').index
tagged_id = test_tagged_users_categorical_features['id'].fillna('')
corpus    = test_tagged_users_categorical_features['tagged_users'].fillna('')

#transform
tf_idf_data = tf_idf_vec.transform(corpus)

#create dataframe
test_tf_idf_tagged=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
test_tf_idf_tagged.set_index(test_tagged_index)
test_tf_idf_tagged['id'] = tagged_id

test_tf_idf_tagged.head()

,1061thetwister,115film,15moferadio,1acd4900c1424d1,1rockstar62,20skyhawkmm20,233liveonline,2for1lapdances,4tiles,4tp_kotc,...,zaynmalik,zenande_mcfen,zeno001,zepp1978,zhenghxn,zojadelin,zomatoaus,zourryart,zxathetis,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11


In [57]:
tagged_index = tagged_users_categorical_features['tagged_users'].fillna('').index
tagged_id = tagged_users_categorical_features['id'].fillna('')
corpus = tagged_users_categorical_features['tagged_users'].fillna('')

# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_tagged=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_smooth_tagged.set_index(tagged_index)
df_tf_idf_smooth_tagged['id'] = tagged_id

df_tf_idf_smooth_tagged.head()

,1061thetwister,115film,15moferadio,1acd4900c1424d1,1rockstar62,20skyhawkmm20,233liveonline,2for1lapdances,4tiles,4tp_kotc,...,zaynmalik,zenande_mcfen,zeno001,zepp1978,zhenghxn,zojadelin,zomatoaus,zourryart,zxathetis,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [58]:
test_tagged_index = test_tagged_users_categorical_features['tagged_users'].fillna('').index
tagged_id = test_tagged_users_categorical_features['id'].fillna('')
corpus    = test_tagged_users_categorical_features['tagged_users'].fillna('')

#transform
tf_idf_data = tf_idf_vec.transform(corpus)

#create dataframe
test_tf_idf_smooth_tagged=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
test_tf_idf_smooth_tagged.set_index(test_tagged_index)
test_tf_idf_smooth_tagged['id'] = tagged_id

test_tf_idf_smooth_tagged.head()

,1061thetwister,115film,15moferadio,1acd4900c1424d1,1rockstar62,20skyhawkmm20,233liveonline,2for1lapdances,4tiles,4tp_kotc,...,zaynmalik,zenande_mcfen,zeno001,zepp1978,zhenghxn,zojadelin,zomatoaus,zourryart,zxathetis,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11


#### Features HASHTAGS

In [59]:
hashtags_numerical_features = df_text.copy()
hashtags_numerical_features = hashtags_agregar_features_numericos(hashtags_numerical_features).drop(columns=['text'])
hashtags_categorical_features = df_text.copy()
hashtags_categorical_features = hashtags_agregar_features_categoricos(hashtags_categorical_features).drop(columns=['text'])

In [60]:
test_hashtags_numerical_features = df_test_text.copy()
test_hashtags_numerical_features = hashtags_agregar_features_numericos(test_hashtags_numerical_features).drop(columns=['text'])
test_hashtags_categorical_features = df_test_text.copy()
test_hashtags_categorical_features = hashtags_agregar_features_categoricos(test_hashtags_categorical_features).drop(columns=['text'])

In [61]:
test_hashtags_categorical_features.head()

,id,hashtags
0,0,[]
1,2,[earthquake]
2,3,[]
3,9,"[Spokane, wildfires]"
4,11,[]


#### TF-IDF - HASHTAGS

In [62]:
hashtags_categorical_features['hashtags'] = hashtags_categorical_features['hashtags'].apply(lambda x:toText(x))
hashtag_index = hashtags_categorical_features['hashtags'].fillna('').index
hashtag_id = hashtags_categorical_features['id'].fillna('')
corpus = hashtags_categorical_features['hashtags'].fillna('')

# Without Smoothing:
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_hashtags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_hashtags.set_index(hashtag_index)
df_tf_idf_hashtags['id'] = hashtag_id

df_tf_idf_hashtags.head()

,0215,034,039,06,09,1008pla_,1008planet_,124,140,16,...,yzf,zaatari,zimmerman,zionism,zionist,ziphimup,zippednews,zippoline,zombiefunrun2014,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [63]:
test_hashtags_categorical_features['hashtags'] = test_hashtags_categorical_features['hashtags'].apply(lambda x:toText(x))
hashtag_index = test_hashtags_categorical_features['hashtags'].fillna('').index
hashtag_id = test_hashtags_categorical_features['id'].fillna('')
corpus = test_hashtags_categorical_features['hashtags'].fillna('')

#transform
tf_idf_data = tf_idf_vec.transform(corpus)

#create dataframe
test_df_tf_idf_hashtags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
test_df_tf_idf_hashtags.set_index(hashtag_index)
test_df_tf_idf_hashtags['id'] = hashtag_id

test_df_tf_idf_hashtags.head()

,0215,034,039,06,09,1008pla_,1008planet_,124,140,16,...,yzf,zaatari,zimmerman,zionism,zionist,ziphimup,zippednews,zippoline,zombiefunrun2014,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11


In [64]:
hashtag_index = hashtags_categorical_features['hashtags'].fillna('').index
hashtag_id = hashtags_categorical_features['id'].fillna('')
corpus = hashtags_categorical_features['hashtags'].fillna('')

# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_hashtags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_smooth_hashtags.set_index(hashtag_index)
df_tf_idf_smooth_hashtags['id'] = hashtag_id

df_tf_idf_smooth_hashtags.head()

,0215,034,039,06,09,1008pla_,1008planet_,124,140,16,...,yzf,zaatari,zimmerman,zionism,zionist,ziphimup,zippednews,zippoline,zombiefunrun2014,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [65]:
hashtag_index = test_hashtags_categorical_features['hashtags'].fillna('').index
hashtag_id = test_hashtags_categorical_features['id'].fillna('')
corpus = test_hashtags_categorical_features['hashtags'].fillna('')

#transform
tf_idf_data = tf_idf_vec.transform(corpus)

#create dataframe
test_df_tf_idf_smooth_hashtags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
test_df_tf_idf_smooth_hashtags.set_index(hashtag_index)
test_df_tf_idf_smooth_hashtags['id'] = hashtag_id

test_df_tf_idf_smooth_hashtags.head()

,0215,034,039,06,09,1008pla_,1008planet_,124,140,16,...,yzf,zaatari,zimmerman,zionism,zionist,ziphimup,zippednews,zippoline,zombiefunrun2014,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11


### Pronombres

In [66]:
def extract_pronoun(tweet, words):
    res = ""
    for w in tweet.lower().split(" "):
        if w in words:
            res+= " " + words[0]
    return res

In [254]:
df_pronombres = train[['id']].copy()


df_pronombres['she'] = train['text'].apply(lambda x: extract_pronoun(x, ["she", "she's"]))
df_pronombres['he'] = train['text'].apply(lambda x: extract_pronoun(x, ["he", "he's"]))
df_pronombres['it'] = train['text'].apply(lambda x: extract_pronoun(x, ["it", "it's"]))
df_pronombres['i'] = train['text'].apply(lambda x: extract_pronoun(x, ["i", "i'm"]))
df_pronombres['you'] = train['text'].apply(lambda x: extract_pronoun(x, ["you","you're"]))
df_pronombres['they'] = train['text'].apply(lambda x: extract_pronoun(x, ["they","they're"]))
df_pronombres['we'] = train['text'].apply(lambda x: extract_pronoun(x, ["we","we're"]))

df_pronombres['pronombres'] = df_pronombres['she'] + df_pronombres['he']+ df_pronombres['they']+\
            df_pronombres['it'] +df_pronombres['you'] + df_pronombres['i'] + df_pronombres['we'] 
df_pronombres['#pronombres'] = df_pronombres['pronombres'].apply(lambda x: len(x.split(" "))-1)

In [69]:
corpus = df_pronombres['pronombres']
pronombres_id = df_pronombres['id'].fillna('')

# Without Smoothing:
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words=[]) # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
pronombres=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
pronombres['id'] = pronombres_id

In [255]:
df_pronombres_test = test[['id']].copy()


df_pronombres_test['she'] = test['text'].apply(lambda x: extract_pronoun(x, ["she", "she's"]))
df_pronombres_test['he'] = test['text'].apply(lambda x: extract_pronoun(x, ["he", "he's"]))
df_pronombres_test['it'] = test['text'].apply(lambda x: extract_pronoun(x, ["it", "it's"]))
df_pronombres_test['i'] = test['text'].apply(lambda x: extract_pronoun(x, ["i", "i'm"]))
df_pronombres_test['you'] = test['text'].apply(lambda x: extract_pronoun(x, ["you","you're"]))
df_pronombres_test['they'] = test['text'].apply(lambda x: extract_pronoun(x, ["they","they're"]))
df_pronombres_test['we'] = test['text'].apply(lambda x: extract_pronoun(x, ["we","we're"]))

df_pronombres_test['pronombres'] = df_pronombres_test['she'] + df_pronombres_test['he']+ df_pronombres_test['they']+\
            df_pronombres_test['it'] +df_pronombres_test['you'] + df_pronombres_test['i'] + df_pronombres_test['we']

df_pronombres_test['#pronombres'] = df_pronombres_test['pronombres'].apply(lambda x: len(x.split(" "))-1)

In [71]:
corpus = df_pronombres_test['pronombres']
pronombres_id = df_pronombres['id'].fillna('')

# Without Smoothing:
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words=[]) # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
pronombres_test=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
pronombres_test['id'] = pronombres_id

#### pos tag

In [72]:
from nltk import word_tokenize, pos_tag

def get_pos_tag(sentence):
    res = ""
    pos_tags =  pos_tag(word_tokenize(sentence))
    for tag in pos_tags:
        res+=tag[1]+" "
    return res

In [73]:
df_pos_tag = train[['id']].copy()

df_pos_tag['pos_tag'] = train['text'].apply(lambda x: get_pos_tag(x))

In [74]:
df_pos_tag.head()

,id,pos_tag
0,1,PRP$ NNS VBP DT NNP IN DT # NN NNP NNP NNP PRP...
1,4,NNP NN IN NNP NNP NNP . NNP
2,5,DT NNS VBD TO VB IN NN '' VBP VBG VBN IN NNS ....
3,6,CD NNS JJ # NNS VBP NNS IN NNP
4,7,RB VBN VBD DT NN IN NNP # NNP IN NN IN # NNS V...


In [75]:
corpus = df_pos_tag['pos_tag']
pos_tag_id = df_pos_tag['id'].fillna('')

# Without Smoothing:
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words=[]) # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
pos_tag_train=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
pos_tag_train['id'] = pos_tag_id

In [76]:
df_pos_tag_test = test[['id']].copy()

In [77]:
df_pos_tag_test['pos_tag'] = test['text'].apply(lambda x: get_pos_tag(x))

In [78]:
corpus = df_pos_tag_test['pos_tag']
pos_tag_id = df_pos_tag_test['id'].fillna('')

# Without Smoothing:
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words=[]) # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
pos_tag_test = pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
pos_tag_test['id'] = pos_tag_id

Exportacion de Dfs

In [79]:
#df_tf_idf_tagged.to_csv('features/train/features_tagged.csv', encoding='utf-8')
#df_tf_idf_smooth_tagged.to_csv('features/train/features_tagged_smooth.csv', encoding='utf-8')
#df_tf_idf_hashtags.to_csv('features/train/features_hashtags.csv', encoding='utf-8')
#df_tf_idf_smooth_hashtags.to_csv('features/train/features_hashtags_smooth.csv', encoding='utf-8')
#tagged_users_numerical_features.to_csv('features/train/features_arrobas_numerical.csv', encoding='utf-8')
#text_numerical_features.to_csv('features/train/features_text_numerical.csv', encoding='utf-8')
#hashtags_numerical_features.to_csv('features/train/features_hashtags_numerical.csv', encoding='utf-8')
# pronombres.to_csv('features/train/features_pronombres.csv', encoding='utf-8')
#pos_tag_train.to_csv('features/train/feature_pos_tag.csv', encoding='utf-8')

In [80]:
#test_tf_idf_tagged.to_csv('features/test/features_tagged.csv', encoding='utf-8')
#test_tf_idf_smooth_tagged.to_csv('features/test/features_tagged_smooth.csv', encoding='utf-8')
#test_df_tf_idf_hashtags.to_csv('features/test/features_hashtags.csv', encoding='utf-8')
#test_df_tf_idf_smooth_hashtags.to_csv('features/test/features_hashtags_smooth.csv', encoding='utf-8')
#test_tagged_users_numerical_features.to_csv('features/test/features_arrobas_numerical.csv', encoding='utf-8')
#test_text_numerical_features.to_csv('features/test/features_text_numerical.csv', encoding='utf-8')
#test_hashtags_numerical_features.to_csv('features/test/features_hashtags_numerical.csv', encoding='utf-8')
#pronombres_test.to_csv('features/test/features_pronombres.csv', encoding='utf-8')
#pos_tag_test.to_csv('features/test/feature_pos_tag.csv', encoding='utf-8')

## 2.3 Location

In [81]:
def agregar_features_location_in_text(df):
    df["cities"]       = df["text"].apply(lambda x: GeoText(str(x)).cities)
    df["cities_str"]   = df["cities"].apply(lambda x: to_string(x,","))
    df["country"]      = df["text"].apply(lambda x: get_country(x))
    df["counties"]     = df["text"].apply(lambda x: check_if_US_county(str(x)))
    df["counties_str"] = df["counties"].apply(lambda x: to_string(x,","))
    df["states"]       = df["text"].apply(lambda x: check_if_US_state(str(x)))
    
    df.loc[(df['states'].str.len() == 0), 'states'] = df.counties.apply(lambda x: get_US_state(x))

    df["country_str"]  = df["country"].apply(lambda x: to_string(x,","))
    df["states_str"]   = df["states"].apply(lambda x: to_string(x,","))
      
    df.loc[~pd.isnull(df['counties_str']), 'country_str'] = 'US'
    df.loc[~pd.isnull(df['states_str']), 'country_str']   = 'US'

    df["continent"]    = df["text"].apply(lambda x: check_if_continent(str(x)))
    df.loc[df['continent'].str.len() == 0, 'continent'] = df.country_str.apply(lambda x: get_continent(x))

    df["continent_str"] = df["continent"].apply(lambda x: to_string(x,","))
    df["has_country"]   = (np.where(pd.isnull(df['country_str']), False, True)).astype(int)
    df["has_city"]      = (np.where(pd.isnull(df['cities_str']), False, True)).astype(int)
    df["has_county"]    = (np.where(pd.isnull(df['counties_str']), False, True)).astype(int)
    df["has_state"]     = (np.where(pd.isnull(df['states_str']), False, True)).astype(int)
    df["has_continent"] = (np.where(pd.isnull(df['continent_str']), False, True)).astype(int)
    return df

def agregar_features_location(df):
    df["has_location"] = (df["location"] != "").astype(int)
    df["cities"]       = df["location"].apply(lambda x: GeoText(str(x)).cities)
    df["cities_str"]   = df["cities"].apply(lambda x: to_string(x,","))
    df["country"]      = df["location"].apply(lambda x: get_country(x))
    df["counties"]     = df["location"].apply(lambda x: check_if_US_county(str(x)))
    df["counties_str"] = df["counties"].apply(lambda x: to_string(x,","))
    df["states"]       = df["location"].apply(lambda x: check_if_US_state(str(x)))
    df.loc[(df['states'].str.len() == 0), 'states'] = df.counties.apply(lambda x: get_US_state(x))
    df["country_str"]  = df["country"].apply(lambda x: to_string(x,","))
    df["states_str"]   = df["states"].apply(lambda x: to_string(x,","))
    df.loc[~pd.isnull(df['counties_str']), 'country_str'] = 'US'
    df.loc[~pd.isnull(df['states_str']), 'country_str']   = 'US'
    df["continent"]    = df["location"].apply(lambda x: check_if_continent(str(x)))
    df.loc[df['continent'].str.len() == 0, 'continent']   = df.country_str.apply(lambda x: get_continent(x))
    df["continent_str"] = df["continent"].apply(lambda x: to_string(x,","))
    df["has_country"]   = (np.where(pd.isnull(df['country_str']), False, True)).astype(int)
    df["has_city"]      = (np.where(pd.isnull(df['cities_str']), False, True)).astype(int)
    df["has_county"]    = (np.where(pd.isnull(df['counties_str']), False, True)).astype(int)
    df["has_state"]     = (np.where(pd.isnull(df['states_str']), False, True)).astype(int)
    df["has_continent"] = (np.where(pd.isnull(df['continent_str']), False, True)).astype(int)
    return df

### Features LOCATION IN TEXT

In [82]:
train_location_in_text = train[["id", "text"]].copy()
train_location_in_text['text'] = train_location_in_text['text'].apply(lambda x: limpiar_tweet_location(x))
train_location_in_text = agregar_features_location_in_text(train_location_in_text)

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [83]:
test_location_in_text = test[["id", "text"]].copy()
test_location_in_text['text'] = test_location_in_text['text'].apply(lambda x: limpiar_tweet_location(x))
test_location_in_text = agregar_features_location_in_text(test_location_in_text)

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [84]:
train_location_in_text_numerical = train_location_in_text.loc[:,["id","target","has_country","has_city","has_county","has_state","has_continent"]]
test_location_in_text_numerical  = test_location_in_text.loc[:,["id","has_country","has_city","has_county","has_state","has_continent"]]

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


#### Encoding de variables categóricas

In [85]:
location_in_text = train_location_in_text.copy()
train_location_in_text_country = location_in_text.loc[:,['id','country']].join(pd.DataFrame(mlb.fit_transform(location_in_text.pop('country')),
                          columns=mlb.classes_,
                          index=location_in_text.index).add_prefix('country_')).drop(columns=['country'])

location_in_text_test = test_location_in_text.copy()
test_location_in_text_country = location_in_text_test.loc[:,['id','country']].join(pd.DataFrame(mlb.transform(location_in_text_test.pop('country')),
                          columns=mlb.classes_,
                          index=location_in_text_test.index).add_prefix('country_')).drop(columns=['country'])

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['AL', 'ER', 'FJ', 'GH', 'GY', 'HT', 'KR', 'MV', 'QA', 'SG', 'SL', 'TL', 'TM'] will be ignored
  .format(sorted(unknown, key=str)))


In [86]:
#location_in_text = train_location_in_text.copy()
train_location_in_text_city = location_in_text.loc[:,['id','cities']].join(pd.DataFrame(mlb.fit_transform(location_in_text.pop('cities')),
                          columns=mlb.classes_,
                          index=location_in_text.index).add_prefix('city_')).drop(columns=['cities'])

test_location_in_text_city = location_in_text_test.loc[:,['id','cities']].join(pd.DataFrame(mlb.transform(location_in_text_test.pop('cities')),
                          columns=mlb.classes_,
                          index=location_in_text_test.index).add_prefix('city_')).drop(columns=['cities'])

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['Agen', 'Amsterdam', 'Asia', 'Bell', 'Bethesda', 'Boo', 'Cape Town', 'Clinton', 'Coney Island', 'Cornwall', 'Dome', 'Fall River', 'Forest Grove', 'Foster', 'Gaza', 'Gresham', 'Grove', 'Hale', 'Harare', 'Hyderabad', 'Isla Vista', 'Issaquah', 'Johnson', 'Kano', 'Keller', 'Kolkata', 'Liberty', 'Liverpool', 'Long Island', 'Male', 'March', 'Marion', 'Mary', 'Newcastle', 'Raleigh', 'Rossendale', 'Same', 'Sterling', 'West Raleigh'] will be ignored
  .format(sorted(unknown, key=str)))


In [87]:
#location_in_text = train_location_in_text.copy()
train_location_in_text_county = location_in_text.loc[:,['id','counties']].join(pd.DataFrame(mlb.fit_transform(location_in_text.pop('counties')),
                          columns=mlb.classes_,
                          index=location_in_text.index).add_prefix('county_')).drop(columns=['counties'])

test_location_in_text_county = location_in_text_test.loc[:,['id','counties']].join(pd.DataFrame(mlb.transform(location_in_text_test.pop('counties')),
                          columns=mlb.classes_,
                          index=location_in_text_test.index).add_prefix('county_')).drop(columns=['counties'])

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['Avery County', 'Burlington County', 'Jackson County'] will be ignored
  .format(sorted(unknown, key=str)))


In [88]:
#location_in_text = train_location_in_text.copy()
train_location_in_text_state = location_in_text.loc[:,['id','states']].join(pd.DataFrame(mlb.fit_transform(location_in_text.pop('states')),
                          columns=mlb.classes_,
                          index=location_in_text.index).add_prefix('state_')).drop(columns=['states'])

test_location_in_text_state = location_in_text_test.loc[:,['id','states']].join(pd.DataFrame(mlb.transform(location_in_text_test.pop('states')),
                          columns=mlb.classes_,
                          index=location_in_text_test.index).add_prefix('state_')).drop(columns=['states'])

In [89]:
#location_in_text = train_location_in_text.copy()
train_location_in_text_continent = location_in_text.loc[:,['id','continent']].join(pd.DataFrame(mlb.fit_transform(location_in_text.pop('continent')),
                          columns=mlb.classes_,
                          index=location_in_text.index).add_prefix('continent_')).drop(columns=['continent'])

test_location_in_text_continent = location_in_text_test.loc[:,['id','continent']].join(pd.DataFrame(mlb.transform(location_in_text_test.pop('continent')),
                          columns=mlb.classes_,
                          index=location_in_text_test.index).add_prefix('continent_')).drop(columns=['continent'])

#### Exportación dataframes

In [90]:
#EXPORTAR CSV
'''
train_location_in_text_numerical.to_csv('features/train/features_location_in_text_numerical.csv', encoding='utf-8')
test_location_in_text_numerical.to_csv('features/test/features_location_in_text_numerical.csv', encoding='utf-8')
train_location_in_text_country.to_csv('features/train/features_country_in_text_encoded.csv', encoding='utf-8')
test_location_in_text_country.to_csv('features/test/features_country_in_text_encoded.csv', encoding='utf-8')
train_location_in_text_city.to_csv('features/train/features_city_in_text_encoded.csv', encoding='utf-8')
test_location_in_text_city.to_csv('features/test/features_city_in_text_encoded.csv', encoding='utf-8')
train_location_in_text_county.to_csv('features/train/features_county_in_text_encoded.csv', encoding='utf-8')
test_location_in_text_county.to_csv('features/test/features_county_in_text_encoded.csv', encoding='utf-8')
train_location_in_text_state.to_csv('features/train/features_state_in_text_encoded.csv', encoding='utf-8')
test_location_in_text_state.to_csv('features/test/features_state_in_text_encoded.csv', encoding='utf-8')
train_location_in_text_continent.to_csv('features/train/features_continent_in_text_encoded.csv', encoding='utf-8')
test_location_in_text_continent.to_csv('features/test/features_continent_in_text_encoded.csv', encoding='utf-8')
'''

"\ntrain_location_in_text_numerical.to_csv('features/train/features_location_in_text_numerical.csv', encoding='utf-8')\ntest_location_in_text_numerical.to_csv('features/test/features_location_in_text_numerical.csv', encoding='utf-8')\ntrain_location_in_text_country.to_csv('features/train/features_country_in_text_encoded.csv', encoding='utf-8')\ntest_location_in_text_country.to_csv('features/test/features_country_in_text_encoded.csv', encoding='utf-8')\ntrain_location_in_text_city.to_csv('features/train/features_city_in_text_encoded.csv', encoding='utf-8')\ntest_location_in_text_city.to_csv('features/test/features_city_in_text_encoded.csv', encoding='utf-8')\ntrain_location_in_text_county.to_csv('features/train/features_county_in_text_encoded.csv', encoding='utf-8')\ntest_location_in_text_county.to_csv('features/test/features_county_in_text_encoded.csv', encoding='utf-8')\ntrain_location_in_text_state.to_csv('features/train/features_state_in_text_encoded.csv', encoding='utf-8')\ntest_lo

### Features LOCATION

In [91]:
train_location = train[["id", "location","target"]].copy().fillna("")
train_location["location"] = train_location["location"].apply(lambda x: limpiar_tweet_location(x))

In [92]:
test_location = test[["id", "location"]].copy().fillna("")
test_location["location"] = test_location["location"].apply(lambda x: limpiar_tweet_location(x))

In [93]:
train_location = agregar_features_location(train_location)
test_location = agregar_features_location(test_location)

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [94]:
train_location_numerical = train_location.loc[:,["id", "target","has_country","has_city","has_county","has_state","has_continent"]]
test_location_numerical  = test_location.loc[:,["id","has_country","has_city","has_county","has_state","has_continent"]]

#### Encoding de variables categóricas

In [95]:
location = train_location.copy()
train_location_country = location.loc[:,['id','country']].join(pd.DataFrame(mlb.fit_transform(location.pop('country')),
                          columns=mlb.classes_,
                          index=location.index).add_prefix('country_')).drop(columns=['country'])

location_test = test_location.copy()
test_location_country = location_test.loc[:,['id','country']].join(pd.DataFrame(mlb.transform(location_test.pop('country')),
                          columns=mlb.classes_,
                          index=location_test.index).add_prefix('country_')).drop(columns=['country'])

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['BS', 'GY', 'KP', 'LS', 'MM', 'MT', 'NP', 'PA', 'TN', 'YE'] will be ignored
  .format(sorted(unknown, key=str)))


In [96]:
#location_in_text = train_location_in_text.copy()
train_location_city = location.loc[:,['id','cities']].join(pd.DataFrame(mlb.fit_transform(location.pop('cities')),
                          columns=mlb.classes_,
                          index=location.index).add_prefix('city_')).drop(columns=['cities'])

test_location_city = location_test.loc[:,['id','cities']].join(pd.DataFrame(mlb.transform(location_test.pop('cities')),
                          columns=mlb.classes_,
                          index=location_test.index).add_prefix('city_')).drop(columns=['cities'])

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['Algonquin', 'Amman', 'Ankeny', 'Balikpapan', 'Barnsley', 'Bellingham', 'Berkeley', 'Boca Raton', 'Bowling Green', 'Brussels', 'Cairo', 'Calabasas', 'Cape Coral', 'Carrboro', 'Carthage', 'Cary', 'Cedar City', 'Cheltenham', 'Chillicothe', 'Clearlake', 'Conway', 'Courtenay', 'Dalton', 'Davis', 'Deerfield Beach', 'Edmonton', 'Edwardsville', 'Eugene', 'Fano', 'Fargo', 'Fayetteville', 'Florence', 'Forest Grove', 'Fredericksburg', 'Fredericton', 'Goldsboro', 'Green', 'Greer', 'Gresham', 'Gurgaon', 'Hannover', 'Harare', 'Hartford', 'Highlands Ranch', 'Homewood', 'Hudson', 'Hull', 'Huntington Beach', 'Hutchinson', 'Indian Trail', 'Irvine', 'Issaquah', 'Jabalpur', 'Jefferson', 'Johnstown', 'Kearney', 'Keffi', 'Lake Charles', 'Lake Worth', 'Lakeland', 'Lansing', 'Laporte', 'Lawrence', 'Lehigh Acres', 'Lemon Grove', 'Lille', 'Lucknow', 'Mechanicsville', 'Midland', 'Mila

In [97]:
#location_in_text = train_location_in_text.copy()
train_location_county = location.loc[:,['id','counties']].join(pd.DataFrame(mlb.fit_transform(location.pop('counties')),
                          columns=mlb.classes_,
                          index=location.index).add_prefix('county_')).drop(columns=['counties'])

test_location_county = location_test.loc[:,['id','counties']].join(pd.DataFrame(mlb.transform(location_test.pop('counties')),
                          columns=mlb.classes_,
                          index=location_test.index).add_prefix('county_')).drop(columns=['counties'])

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['Dade County', 'Johnson County', 'Ventura County'] will be ignored
  .format(sorted(unknown, key=str)))


In [98]:
#location_in_text = train_location_in_text.copy()
train_location_state = location.loc[:,['id','states']].join(pd.DataFrame(mlb.fit_transform(location.pop('states')),
                          columns=mlb.classes_,
                          index=location.index).add_prefix('state_')).drop(columns=['states'])

test_location_state = location_test.loc[:,['id','states']].join(pd.DataFrame(mlb.transform(location_test.pop('states')),
                          columns=mlb.classes_,
                          index=location_test.index).add_prefix('state_')).drop(columns=['states'])

In [99]:
train_location_continent = location.loc[:,['id','continent']].join(pd.DataFrame(mlb.fit_transform(location.pop('continent')),
                          columns=mlb.classes_,
                          index=location.index).add_prefix('continent_')).drop(columns=['continent'])

test_location_continent = location_test.loc[:,['id','continent']].join(pd.DataFrame(mlb.transform(location_test.pop('continent')),
                          columns=mlb.classes_,
                          index=location_test.index).add_prefix('continent_')).drop(columns=['continent'])

In [100]:
#EXPORTAR CSV
'''train_location_numerical.to_csv('features/train/features_location_numerical.csv', encoding='utf-8')
test_location_numerical.to_csv('features/test/features_location_numerical.csv', encoding='utf-8')
train_location_country.to_csv('features/train/features_country_encoded.csv', encoding='utf-8')
test_location_country.to_csv('features/test/features_country_encoded.csv', encoding='utf-8')
train_location_city.to_csv('features/train/features_city_encoded.csv', encoding='utf-8')
test_location_city.to_csv('features/test/features_city_encoded.csv', encoding='utf-8')
train_location_county.to_csv('features/train/features_county_encoded.csv', encoding='utf-8')
test_location_county.to_csv('features/test/features_county_encoded.csv', encoding='utf-8')
train_location_state.to_csv('features/train/features_state_encoded.csv', encoding='utf-8')
test_location_state.to_csv('features/test/features_state_encoded.csv', encoding='utf-8')
train_location_continent.to_csv('features/train/features_continent_encoded.csv', encoding='utf-8')
test_location_continent.to_csv('features/test/features_continent_encoded.csv', encoding='utf-8')'''

"train_location_numerical.to_csv('features/train/features_location_numerical.csv', encoding='utf-8')\ntest_location_numerical.to_csv('features/test/features_location_numerical.csv', encoding='utf-8')\ntrain_location_country.to_csv('features/train/features_country_encoded.csv', encoding='utf-8')\ntest_location_country.to_csv('features/test/features_country_encoded.csv', encoding='utf-8')\ntrain_location_city.to_csv('features/train/features_city_encoded.csv', encoding='utf-8')\ntest_location_city.to_csv('features/test/features_city_encoded.csv', encoding='utf-8')\ntrain_location_county.to_csv('features/train/features_county_encoded.csv', encoding='utf-8')\ntest_location_county.to_csv('features/test/features_county_encoded.csv', encoding='utf-8')\ntrain_location_state.to_csv('features/train/features_state_encoded.csv', encoding='utf-8')\ntest_location_state.to_csv('features/test/features_state_encoded.csv', encoding='utf-8')\ntrain_location_continent.to_csv('features/train/features_contin

### Location label countries

In [101]:
# Para aquellos tweets con más de dos paises, separo en columnas

countries = location['country_str'].str.split(",", expand = True)

countries["id"] = location["id"]
countries.set_index("id")

countries1 = pd.get_dummies(countries[0],"Country")
countries2 = pd.get_dummies(countries[1],"Country")

# Uno las columnas
for country in countries1.columns:
    if(country in countries2.columns):
        countries1[country] = countries1[country] + countries2[country]
for country in countries2.columns:
    if(country in countries1.columns):
        countries1[country] = countries2[country]

In [102]:
countries1.head(2)

,Country_AE,Country_AF,Country_AQ,Country_AR,Country_AT,Country_AU,Country_BB,Country_BD,Country_BE,Country_BG,...,Country_TH,Country_TM,Country_UA,Country_UG,Country_US,Country_UY,Country_VE,Country_VN,Country_ZA,Country_ZW
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### BOW: Coutry

In [103]:
#bow_country_index = location['country_str'].fillna('').index
corpus = list(location['country_str'].fillna(''))

In [104]:
country_vectorizer = CountVectorizer()

bow_country = country_vectorizer.fit_transform(corpus).todense()
df_bow_country = pd.DataFrame(bow_country)
df_bow_country = df_bow_country.set_index(location['id'].index)
df_bow_country.columns = ['country_'+ c for c in  country_vectorizer.get_feature_names()]

In [105]:
df_bow_country.head()

,country_ae,country_af,country_aq,country_ar,country_at,country_au,country_bb,country_bd,country_be,country_bg,...,country_th,country_tm,country_ua,country_ug,country_us,country_uy,country_ve,country_vn,country_za,country_zw
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Tf-Idf: Country

In [106]:
# Without Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_country=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_country.columns = ['country_'+ c for c in df_tf_idf_country.columns]
df_tf_idf_country = df_tf_idf_country.set_index(location['id'].index)

df_tf_idf_country.head()

,country_ae,country_af,country_aq,country_ar,country_au,country_bb,country_bd,country_bg,country_bh,country_br,...,country_sy,country_th,country_tm,country_ua,country_ug,country_uy,country_ve,country_vn,country_za,country_zw
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
column_names_c = ['country_'+ c for c in  tf_idf_vec.get_feature_names()]
df_tf_idf_smooth_country=pd.DataFrame(tf_idf_data.toarray(),columns=column_names_c)
df_tf_idf_smooth_country.set_index(location['id'].index)

df_tf_idf_smooth_country.head()

,country_ae,country_af,country_aq,country_ar,country_au,country_bb,country_bd,country_bg,country_bh,country_br,...,country_sy,country_th,country_tm,country_ua,country_ug,country_uy,country_ve,country_vn,country_za,country_zw
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
nueva_posicion= df_bow_country.columns.get_loc('country_ae')
df_bow_country.insert(loc= nueva_posicion, column='id', value = df_bow_country.index)

In [109]:
df_tf_idf_smooth_country.insert(loc= nueva_posicion, column='id', value = df_tf_idf_smooth_country.index)

In [110]:
# DOWNLOAD CSV LOCATION

# TRAIN

#df_tf_idf_smooth_country.to_csv('features/train/features_location_tf_idf.csv', encoding='utf-8')
#df_bow_country.to_csv('features/train/features_location_bow.csv', encoding='utf-8')

## 2.4 Links

In [111]:
def agregar_features_links(df):

    df["links_domain"] = df["http"].apply(lambda x: get_dominio(x))
    df["links_encription"] = df["http"].apply(lambda x: type_http(x))
    df["links_cant"] = df["links_cant"]
    df["links_encription_type"] = df["links_encription"].apply(lambda x: link_encription_generalize(x))
    df["cant_failed_links"] = df["http"].apply(lambda x: count_None(x))
    df["links_domain_str"] = df["links_domain"].apply(lambda x: to_string(x,","))
    return df

In [112]:
links_train = agregar_features_links(links_train)
links_test = agregar_features_links(links_test)

In [113]:
features_links_train_numerical = links_train.loc[:,['target','id','links_cant','cant_failed_links']]
features_links_test_numerical = links_test.loc[:,['target','id','links_cant','cant_failed_links']]

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


### Dominios train

In [114]:
domain_train_index = links_train['links_domain_str'].dropna().index
corpus = list(links_train['links_domain_str'].dropna())

#### BOW

In [115]:
domain_vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = ["www", "com"],   \
                             max_features = 5000)

bow_domain_train = domain_vectorizer.fit_transform(corpus).todense()
df_bow_domain_train = pd.DataFrame(bow_domain_train)
df_bow_domain_train = df_bow_domain_train.set_index(domain_train_index)
df_bow_domain_train.columns = ["domain_" + c for c in domain_vectorizer.get_feature_names()]
df_bow_domain_train = df_bow_domain_train.reset_index()

df_bow_domain_train.head()

,id,domain_10news,domain_1stheadlines,domain_24x7freeadvice,domain_28chronicle,domain_28national,domain_28ndtv,domain_28zero,domain_29,domain_2c,...,domain_zalkus,domain_zamanalwsl,domain_zeenews,domain_zenit,domain_zeppy,domain_zero,domain_zerohedge,domain_ziprecruiter,domain_zoma,domain_zw
0,48,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,50,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,53,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Tf-Idf

In [116]:
# Without Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe

df_tf_idf_domain_train_cols = ["domain_" + c for c in tf_idf_vec.get_feature_names()]
df_tf_idf_domain_train=pd.DataFrame(tf_idf_data.toarray(),columns=df_tf_idf_domain_train_cols)
df_tf_idf_domain_train = df_tf_idf_domain_train.set_index(domain_train_index)
df_tf_idf_domain_train = df_tf_idf_domain_train.reset_index()

del df_tf_idf_domain_train["domain_www"]
del df_tf_idf_domain_train["domain_com"]

df_tf_idf_domain_train.head()

,id,domain_10news,domain_1stheadlines,domain_24x7freeadvice,domain_28chronicle,domain_28national,domain_28ndtv,domain_28zero,domain_29,domain_2c,...,domain_zalkus,domain_zamanalwsl,domain_zeenews,domain_zenit,domain_zeppy,domain_zero,domain_zerohedge,domain_ziprecruiter,domain_zoma,domain_zw
0,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
# With Smoothing:

tf_idf_vec_smooth = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec_smooth.fit_transform(corpus)

#create dataframe

df_tf_idf_smooth_domain_train_cols = ["domain_" + c for c in tf_idf_vec_smooth.get_feature_names()]
df_tf_idf_smooth_domain_train=pd.DataFrame(tf_idf_data.toarray(),columns=df_tf_idf_domain_train_cols)
df_tf_idf_smooth_domain_train = df_tf_idf_smooth_domain_train.set_index(domain_train_index)
df_tf_idf_smooth_domain_train = df_tf_idf_smooth_domain_train.reset_index()

del df_tf_idf_smooth_domain_train["domain_www"]
del df_tf_idf_smooth_domain_train["domain_com"]

df_tf_idf_smooth_domain_train.head()

,id,domain_10news,domain_1stheadlines,domain_24x7freeadvice,domain_28chronicle,domain_28national,domain_28ndtv,domain_28zero,domain_29,domain_2c,...,domain_zalkus,domain_zamanalwsl,domain_zeenews,domain_zenit,domain_zeppy,domain_zero,domain_zerohedge,domain_ziprecruiter,domain_zoma,domain_zw
0,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Dominios test

In [118]:
domain_test_index = links_test['links_domain_str'].dropna().index
corpus = list(links_test['links_domain_str'].dropna())

#### BOW

In [119]:
bow_domain_test = domain_vectorizer.transform(corpus).todense()
df_bow_domain_test = pd.DataFrame(bow_domain_test)
df_bow_domain_test = df_bow_domain_test.set_index(domain_test_index)
df_bow_domain_test.columns = ["domain_" + c for c in domain_vectorizer.get_feature_names()]
df_bow_domain_test = df_bow_domain_test.reset_index()

df_bow_domain_test.head()

,id,domain_10news,domain_1stheadlines,domain_24x7freeadvice,domain_28chronicle,domain_28national,domain_28ndtv,domain_28zero,domain_29,domain_2c,...,domain_zalkus,domain_zamanalwsl,domain_zeenews,domain_zenit,domain_zeppy,domain_zero,domain_zerohedge,domain_ziprecruiter,domain_zoma,domain_zw
0,46,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,58,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,70,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,75,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,84,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Tf-Idf

In [120]:
#transform
tf_idf_data = tf_idf_vec.transform(corpus)

#create dataframe
df_tf_idf_domain_test_cols = ["domain_" + c for c in tf_idf_vec.get_feature_names()]
df_tf_idf_domain_test = pd.DataFrame(tf_idf_data.toarray(),columns=df_tf_idf_domain_test_cols)
df_tf_idf_domain_test = df_tf_idf_domain_test.set_index(domain_test_index)
df_tf_idf_domain_test = df_tf_idf_domain_test.reset_index()

del df_tf_idf_domain_test["domain_www"]
del df_tf_idf_domain_test["domain_com"]

In [121]:
#transform
tf_idf_data = tf_idf_vec_smooth.transform(corpus)

#create dataframe
df_tf_idf_smooth_domain_test_cols = ["domain_" + c for c in tf_idf_vec_smooth.get_feature_names()]
df_tf_idf_smooth_domain_test = pd.DataFrame(tf_idf_data.toarray(),columns=df_tf_idf_domain_test_cols)
df_tf_idf_smooth_domain_test = df_tf_idf_smooth_domain_test.set_index(domain_test_index)
df_tf_idf_smooth_domain_test = df_tf_idf_smooth_domain_test.reset_index()

del df_tf_idf_smooth_domain_test["domain_www"]
del df_tf_idf_smooth_domain_test["domain_com"]

In [122]:
# DOWNLOAD CSV LINKS

# TRAIN

'''features_links_train_numerical.to_csv("features/train/features_links_numerical.csv", encoding='utf-8')
df_tf_idf_smooth_domain_train.to_csv('features/train/features_domain_smooth_tf_idf.csv', encoding='utf-8')
df_tf_idf_domain_train.to_csv('features/train/features_domain_tf_idf.csv', encoding='utf-8')
df_bow_domain_train.to_csv('features/train/features_domain_bow.csv', encoding='utf-8')


# TEST

features_links_test_numerical.to_csv("features/test/features_links_numerical.csv", encoding='utf-8')
df_tf_idf_smooth_domain_test.to_csv('features/test/features_domain_smooth_tf_idf.csv', encoding='utf-8')
df_tf_idf_domain_test.to_csv('features/test/features_domain_tf_idf.csv', encoding='utf-8')
df_bow_domain_test.to_csv('features/test/features_domain_bow.csv', encoding='utf-8')'''

'features_links_train_numerical.to_csv("features/train/features_links_numerical.csv", encoding=\'utf-8\')\ndf_tf_idf_smooth_domain_train.to_csv(\'features/train/features_domain_smooth_tf_idf.csv\', encoding=\'utf-8\')\ndf_tf_idf_domain_train.to_csv(\'features/train/features_domain_tf_idf.csv\', encoding=\'utf-8\')\ndf_bow_domain_train.to_csv(\'features/train/features_domain_bow.csv\', encoding=\'utf-8\')\n\n\n# TEST\n\nfeatures_links_test_numerical.to_csv("features/test/features_links_numerical.csv", encoding=\'utf-8\')\ndf_tf_idf_smooth_domain_test.to_csv(\'features/test/features_domain_smooth_tf_idf.csv\', encoding=\'utf-8\')\ndf_tf_idf_domain_test.to_csv(\'features/test/features_domain_tf_idf.csv\', encoding=\'utf-8\')\ndf_bow_domain_test.to_csv(\'features/test/features_domain_bow.csv\', encoding=\'utf-8\')'

## 2.5 Tags

### Informacion general tags

In [123]:
# Creo el dataframe
train_tags = train[['id','text','target']].copy(deep=True)

# Limpio el dataframe
train_tags['new'] = train_tags["text"].apply(lambda x: clean_text_strict(x))

In [124]:
# Creo el dataframe
test_tags = test[['id','text']].copy(deep=True)

# Limpio el dataframe
test_tags['new'] = test_tags["text"].apply(lambda x: clean_text_strict(x))

In [125]:
#  Eliminacion de terminos, caracteres y stopwords innecesarios para generar los tags
#convierte de list a string
train_tags['new'] = train_tags['new'].apply(lambda x: [item for item in x.split() if item not in stopwordsList])
train_tags.new = train_tags.new.apply(prettify)

test_tags['new'] = test_tags['new'].apply(lambda x: [item for item in x.split() if item not in stopwordsList])
test_tags.new = test_tags.new.apply(prettify)

In [126]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

train_tags['new_tokenized'] = train_tags.new.apply(lemmatize_text2)
train_tags.new_tokenized = train_tags.new_tokenized.apply(prettify)

test_tags['new_tokenized'] = test_tags.new.apply(lemmatize_text2)
test_tags.new_tokenized = test_tags.new_tokenized.apply(prettify)

In [127]:
a = train_tags['new_tokenized'].apply(getText).str.lower().str.cat(sep=' ')

words = nltk.tokenize.word_tokenize(a)
word_dist = nltk.FreqDist(words)

this_col = pd.Series(['word','frequency'])

rslt = pd.DataFrame(word_dist.most_common(400),columns = this_col)

In [128]:
frequent_words = rslt ['word']
unique_frequent_words = set(frequent_words)

In [129]:
train_tags['tags'] = train_tags.new_tokenized.apply(word_finder)
train_info = train_tags[['id','target','text','tags']]

train_info.head()

,id,target,text,tags
0,1,1,Our Deeds are the Reason of this #earthquake M...,"[reason, earthquake]"
1,4,1,Forest fire near La Ronge Sask. Canada,"[forest, fire]"
2,5,1,All residents asked to 'shelter in place' are ...,"[evacuation, officer]"
3,6,1,"13,000 people receive #wildfires evacuation or...","[evacuation, california, people, wildfire]"
4,7,1,Just got sent this photo from Ruby #Alaska as ...,"[smoke, photo, wildfire]"


In [130]:
a = test_tags['new_tokenized'].apply(getText).str.lower().str.cat(sep=' ')

words = nltk.tokenize.word_tokenize(a)
word_dist = nltk.FreqDist(words)

this_col = pd.Series(['word','frequency'])

rslt = pd.DataFrame(word_dist.most_common(400),columns = this_col)

frequent_words = rslt ['word']
unique_frequent_words = set(frequent_words)

In [131]:
test_tags['tags'] = test_tags.new_tokenized.apply(word_finder)
test_info = test_tags[['id','text','tags']]

test_info.head()

,id,text,tags
0,0,Just happened a terrible car crash,[crash]
1,2,"Heard about #earthquake is different cities, s...","[city, heard, earthquake]"
2,3,"there is a forest fire at spot pond, geese are...","[street, forest, fire]"
3,9,Apocalypse lighting. #Spokane #wildfires,"[apocalypse, wildfire]"
4,11,Typhoon Soudelor kills 28 in China and Taiwan,"[china, kill, typhoon]"


In [132]:
train_info = train_info.copy()
train_info["cant_tags"] = train_info["tags"].apply(lambda x: len(x))
train_info["tags_str"] = train_info["tags"].apply(lambda x: to_string(x,","))

In [133]:
train_info.head(2)

,id,target,text,tags,cant_tags,tags_str
0,1,1,Our Deeds are the Reason of this #earthquake M...,"[reason, earthquake]",2,"earthquake,reason"
1,4,1,Forest fire near La Ronge Sask. Canada,"[forest, fire]",2,"fire,forest"


In [134]:
test_tags['tags'] = test_tags.new_tokenized.apply(word_finder)
test_info = test_tags[['id','text','tags']]
test_info = test_info.copy()
test_info["cant_tags"] = test_info["tags"].apply(lambda x: len(x))
test_info["tags_str"] = test_info["tags"].apply(lambda x: to_string(x,","))

test_info.head()

,id,text,tags,cant_tags,tags_str
0,0,Just happened a terrible car crash,[crash],1,crash
1,2,"Heard about #earthquake is different cities, s...","[city, heard, earthquake]",3,"city,earthquake,heard"
2,3,"there is a forest fire at spot pond, geese are...","[street, forest, fire]",3,"fire,forest,street"
3,9,Apocalypse lighting. #Spokane #wildfires,"[apocalypse, wildfire]",2,"apocalypse,wildfire"
4,11,Typhoon Soudelor kills 28 in China and Taiwan,"[china, kill, typhoon]",3,"china,kill,typhoon"


###  Nuevo feature por tag

In [135]:
tags_columns = train_info['tags_str'].str.split(",", expand = True)

In [136]:
df_has_tags = pd.get_dummies(tags_columns[0],"tag")
unique_tag = set(df_has_tags.columns)

for i in range(1, len(tags_columns.columns)):
    n_links_pd = pd.get_dummies(tags_columns[i],"tag")
    for domain in unique_tag:
        if(domain in set(tags_columns.columns)):
            df_has_tags[domain] = df_has_tags[domain] + tags_columns[domain]

In [137]:
df_has_tags["id"] = train_info["id"]
df_has_tags = df_has_tags.set_index("id")

In [138]:
df_has_tags.head()

,tag_abc,tag_ablaze,tag_accident,tag_action,tag_air,tag_aircraft,tag_airplane,tag_airport,tag_ambulance,tag_america,...,tag_wildfire,tag_win,tag_wind,tag_windstorm,tag_woman,tag_wound,tag_wounded,tag_wreck,tag_wrecked,tag_zone
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
tags_columns = test_info['tags_str'].str.split(",", expand = True)

In [140]:
test_has_tags = pd.get_dummies(tags_columns[0],"tag")
unique_tag = set(test_has_tags.columns)

for i in range(1, len(tags_columns.columns)):
    n_links_pd = pd.get_dummies(tags_columns[i],"tag")
    for domain in unique_tag:
        if(domain in set(tags_columns.columns)):
            test_has_tags[domain] = test_has_tags[domain] + tags_columns[domain]

In [141]:
test_has_tags["id"] = test_info["id"]
test_has_tags = test_has_tags.set_index("id")

### BOW: tags

In [142]:
#bow_tags_index = tags_info['tags_str'].fillna('').index
corpus = list(train_info['tags_str'].fillna(''))

In [143]:
domain_vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

bow_tags = domain_vectorizer.fit_transform(corpus).todense()
df_bow_tags = pd.DataFrame(bow_tags)
df_bow_tags["id"] = train_info["id"]
df_bow_tags = df_bow_tags.set_index("id")
df_bow_tags.columns =["tag_"+c for c in domain_vectorizer.get_feature_names()]

In [144]:
df_bow_tags.head()

,tag_abc,tag_ablaze,tag_accident,tag_action,tag_air,tag_aircraft,tag_airplane,tag_airport,tag_ambulance,tag_america,...,tag_wind,tag_windstorm,tag_woman,tag_wound,tag_wounded,tag_wreck,tag_wreckage,tag_wrecked,tag_yr,tag_zone
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Tf-idf: tags

In [145]:
#bow_tags_index = tags_info['tags_str'].dropna().index
corpus = list(train_info['tags_str'].fillna(''))

In [146]:
# Without Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_tags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_tags["id"] = train_info["id"]
df_tf_idf_tags = df_tf_idf_tags.set_index("id")
df_tf_idf_tags.columns =["tag_"+c for c in tf_idf_vec.get_feature_names()]

df_tf_idf_tags.head()

,tag_abc,tag_ablaze,tag_accident,tag_action,tag_air,tag_aircraft,tag_airplane,tag_airport,tag_ambulance,tag_america,...,tag_wind,tag_windstorm,tag_woman,tag_wound,tag_wounded,tag_wreck,tag_wreckage,tag_wrecked,tag_yr,tag_zone
id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [147]:
# With Smoothing:

tf_idf_vec_smooth = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec_smooth.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_tags =pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec_smooth.get_feature_names())
df_tf_idf_smooth_tags["id"] = train_info["id"]
df_tf_idf_smooth_tags = df_tf_idf_smooth_tags.set_index("id")
df_tf_idf_smooth_tags.columns =["tag_"+c for c in tf_idf_vec_smooth.get_feature_names()]

df_tf_idf_smooth_tags.head()

,tag_abc,tag_ablaze,tag_accident,tag_action,tag_air,tag_aircraft,tag_airplane,tag_airport,tag_ambulance,tag_america,...,tag_wind,tag_windstorm,tag_woman,tag_wound,tag_wounded,tag_wreck,tag_wreckage,tag_wrecked,tag_yr,tag_zone
id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
df_bow_tags=df_bow_tags.reset_index()
df_bow_tags.head()

,id,tag_abc,tag_ablaze,tag_accident,tag_action,tag_air,tag_aircraft,tag_airplane,tag_airport,tag_ambulance,...,tag_wind,tag_windstorm,tag_woman,tag_wound,tag_wounded,tag_wreck,tag_wreckage,tag_wrecked,tag_yr,tag_zone
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
df_tf_idf_smooth_tags = df_tf_idf_smooth_tags.reset_index()
df_tf_idf_smooth_tags.head()

,id,tag_abc,tag_ablaze,tag_accident,tag_action,tag_air,tag_aircraft,tag_airplane,tag_airport,tag_ambulance,...,tag_wind,tag_windstorm,tag_woman,tag_wound,tag_wounded,tag_wreck,tag_wreckage,tag_wrecked,tag_yr,tag_zone
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [150]:
# DOWNLOAD CSV LINKS

# TRAIN

'''df_tf_idf_smooth_tags.to_csv('features/train/features_tags_tf_idf.csv', encoding='utf-8')
df_bow_tags.to_csv('features/train/features_tags_bow.csv', encoding='utf-8')'''


"df_tf_idf_smooth_tags.to_csv('features/train/features_tags_tf_idf.csv', encoding='utf-8')\ndf_bow_tags.to_csv('features/train/features_tags_bow.csv', encoding='utf-8')"

In [151]:
corpus = list(test_info['tags_str'].fillna(''))

In [152]:
#transform
tf_idf_data = tf_idf_vec.transform(corpus)

#create dataframe
test_tf_idf_tags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
test_tf_idf_tags["id"] = test_info["id"]
test_tf_idf_tags = test_tf_idf_tags.set_index("id")
test_tf_idf_tags.columns =["tag_"+c for c in tf_idf_vec.get_feature_names()]

test_tf_idf_tags.head()

,tag_abc,tag_ablaze,tag_accident,tag_action,tag_air,tag_aircraft,tag_airplane,tag_airport,tag_ambulance,tag_america,...,tag_wind,tag_windstorm,tag_woman,tag_wound,tag_wounded,tag_wreck,tag_wreckage,tag_wrecked,tag_yr,tag_zone
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
#transform
tf_idf_data = tf_idf_vec_smooth.transform(corpus)

#create dataframe
test_tf_idf_smooth_tags =pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec_smooth.get_feature_names())
test_tf_idf_smooth_tags["id"] = test_info["id"]
test_tf_idf_smooth_tags = test_tf_idf_smooth_tags.set_index("id")
test_tf_idf_smooth_tags.columns =["tag_"+c for c in tf_idf_vec_smooth.get_feature_names()]

test_tf_idf_smooth_tags.head()

,tag_abc,tag_ablaze,tag_accident,tag_action,tag_air,tag_aircraft,tag_airplane,tag_airport,tag_ambulance,tag_america,...,tag_wind,tag_windstorm,tag_woman,tag_wound,tag_wounded,tag_wreck,tag_wreckage,tag_wrecked,tag_yr,tag_zone
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
bow_tags = domain_vectorizer.transform(corpus).todense()
test_bow_tags = pd.DataFrame(bow_tags)
test_bow_tags["id"] = test_info["id"]
test_bow_tags = test_bow_tags.set_index("id")
test_bow_tags.columns =["tag_"+c for c in domain_vectorizer.get_feature_names()]

In [155]:
test_bow_tags=test_bow_tags.reset_index()
test_tf_idf_smooth_tags = test_tf_idf_smooth_tags.reset_index()

In [156]:
#test_tf_idf_smooth_tags.to_csv('features/test/features_tags_tf_idf.csv', encoding='utf-8')
#test_bow_tags.to_csv('features/test/features_tags_bow.csv', encoding='utf-8')

## Variables categóricas

### Keywords

In [229]:
# train

keywords_categorical_features = train_keywords.copy() 
keywords_categorical_features = keywords_agregar_features_categoricos(keywords_categorical_features,True).drop(columns=['keyword','location','text'])

keywords_numerical_features = train_keywords.copy()
keywords_numerical_features = keywords_agregar_features_numericos(keywords_numerical_features, True).drop(columns=['location','text','keyword'])
del keywords_numerical_features["target"]

In [230]:
keywords_categorical_train = keywords_categorical_features.merge(keywords_numerical_features, on="id", how="left")
keywords_categorical_train.head()

,id,target,keywords_in_text,text_contains_keyword,has_keyword,keywords_quantity,keywords_mean,keyword_is_hashtag,keyword_frequency
0,1,1,[earthquake],1,1,1,0.004525,1,0.227027
1,4,1,"[fire, forest fire]",1,1,1,0.004525,0,0.237838
2,5,1,[evacuation],1,1,1,0.004525,0,0.210811
3,6,1,"[evacuation, fire, wildfire]",1,1,1,0.004525,0,0.210811
4,7,1,"[fire, smoke, wildfire]",1,1,1,0.004525,0,0.237838


In [232]:
# test

keywords_categorical_features = test_keywords.copy() 
keywords_categorical_features = keywords_agregar_features_categoricos(keywords_categorical_features,True).drop(columns=['keyword','location','text'])

keywords_numerical_features = test_keywords.copy()
keywords_numerical_features = keywords_agregar_features_numericos(keywords_numerical_features, True).drop(columns=['location','text','keyword'])

In [234]:
keywords_categorical_test = keywords_categorical_features.merge(keywords_numerical_features, on="id", how="left")
keywords_categorical_test.head()

,id,keywords_in_text,text_contains_keyword,has_keyword,keywords_quantity,keywords_mean,keyword_is_hashtag,keyword_frequency
0,0,[crash],1,1,1,0.004525,0,0.227027
1,2,[earthquake],1,1,1,0.004525,1,0.237838
2,3,"[fire, forest fire]",1,1,1,0.004525,0,0.210811
3,9,"[apocalypse, fire, wildfire]",1,1,1,0.004525,0,0.210811
4,11,[typhoon],1,1,1,0.004525,0,0.237838


### Text

In [282]:
text_categorical_features_train = text_numerical_features
text_categorical_features_train["#pronombres"] = df_pronombres["#pronombres"]
text_categorical_features_train["#tagged_users"] = tagged_users_numerical_features["#tagged_users"]
text_categorical_features_train["tagged_users"] = tagged_users_categorical_features["tagged_users"]
text_categorical_features_train["hashtags"] = hashtags_categorical_features["hashtags"]
text_categorical_features_train["#hashtags"] = hashtags_numerical_features["#hashtags"]
text_categorical_features_train["has_hashtag"] = hashtags_numerical_features["has_hashtag"]
text_categorical_features_train["pos_tag"] = df_pos_tag["pos_tag"]

text_categorical_features_train.head()

,id,target,#palabras,#palabras_unicas,#caracteres,#stopwords,#puntuacion,#capitalize,#mayusculas,#silabas,...,#mayusculas_binned,#silabas_binned,#caracteres_especiales_binned,#tagged_users,#pronombres,tagged_users,hashtags,#hashtags,has_hashtag,pos_tag
0,1,1,13,13,71,5,1,5,1,19,...,0,1,0,0,0,,earthquake,1,1,PRP$ NNS VBP DT NNP IN DT # NN NNP NNP NNP PRP...
1,4,1,7,7,40,0,1,5,0,10,...,0,0,0,0,0,,,0,0,NNP NN IN NNP NNP NNP . NNP
2,5,1,22,20,135,9,3,2,0,27,...,0,3,1,0,0,,,0,0,DT NNS VBD TO VB IN NN '' VBP VBG VBN IN NNS ....
3,6,1,8,8,67,1,2,1,0,17,...,0,1,0,0,0,,wildfires,1,1,CD NNS JJ # NNS VBP NNS IN NNP
4,7,1,16,15,90,6,2,3,0,24,...,0,2,0,0,0,,Alaska wildfires,2,1,RB VBN VBD DT NN IN NNP # NNP IN NN IN # NNS V...


In [291]:
text_categorical_features_test = test_text_numerical_features
text_categorical_features_test["#pronombres"] = df_pronombres_test["#pronombres"]
text_categorical_features_test["#tagged_users"] = test_tagged_users_numerical_features["#tagged_users"]
text_categorical_features_test["tagged_users"] = test_tagged_users_categorical_features["tagged_users"]
text_categorical_features_test["hashtags"] = test_hashtags_categorical_features["hashtags"]
text_categorical_features_test["#hashtags"] = test_hashtags_numerical_features["#hashtags"]
text_categorical_features_test["has_hashtag"] = test_hashtags_numerical_features["has_hashtag"]
text_categorical_features_test["pos_tag"] = df_pos_tag_test["pos_tag"]

text_categorical_features_test.head()

,id,#palabras,#palabras_unicas,#caracteres,#stopwords,#puntuacion,#capitalize,#mayusculas,#silabas,promedio_len_word,...,#mayusculas_binned,#silabas_binned,#caracteres_especiales_binned,#pronombres,#tagged_users,tagged_users,hashtags,#hashtags,has_hashtag,pos_tag
0,0,6,6,36,1,0,1,0,7,4.833333,...,0,0,0,0,0,,,0,0,RB VBD DT JJ NN NN
1,2,9,9,66,2,3,1,0,17,6.222222,...,0,1,1,0,0,,earthquake,1,1,"NNP IN # NN VBZ JJ NNS , VB JJ NN ."
2,3,19,19,98,8,2,1,1,17,4.105263,...,0,1,0,1,0,,,0,0,"EX VBZ DT JJ NN IN NN NN , JJ VBP VBG IN DT NN..."
3,9,4,4,42,0,3,2,0,11,9.250000,...,0,0,1,0,0,,Spokane wildfires,2,1,NNP VBG . # NNP # NNS
4,11,8,8,47,2,0,4,0,15,4.750000,...,0,1,0,0,0,,,0,0,NNP NNP VBZ CD IN NNP CC NNP


### Location

In [189]:
# train

location_categorical_train = train_location[["id","cities_str","counties_str","country_str","states_str", "continent_str","has_location", "has_country", "has_city","has_county", "has_state","has_continent"]]
location_categorical_train.columns = ["id","cities","counties","country","states", "continent","has_location", "has_country", "has_city","has_county", "has_state","has_continent"]
location_categorical_train.head()

,id,cities,counties,country,states,continent,has_location,has_country,has_city,has_county,has_state,has_continent
0,1,None,None,None,None,None,0,0,0,0,0,0
1,4,None,None,None,None,None,0,0,0,0,0,0
2,5,None,None,None,None,None,0,0,0,0,0,0
3,6,None,None,None,None,None,0,0,0,0,0,0
4,7,None,None,None,None,None,0,0,0,0,0,0


In [190]:
# test

location_categorical_test = test_location[["id","cities_str","counties_str","country_str","states_str", "continent_str","has_location", "has_country", "has_city","has_county", "has_state","has_continent"]]
location_categorical_test.columns = ["id","cities","counties","country","states", "continent","has_location", "has_country", "has_city","has_county", "has_state","has_continent"]
location_categorical_test.head()

,id,cities,counties,country,states,continent,has_location,has_country,has_city,has_county,has_state,has_continent
0,0,None,None,None,None,None,0,0,0,0,0,0
1,2,None,None,None,None,None,0,0,0,0,0,0
2,3,None,None,None,None,None,0,0,0,0,0,0
3,9,None,None,None,None,None,0,0,0,0,0,0
4,11,None,None,None,None,None,0,0,0,0,0,0


### Links

In [212]:
# train

links_categorical_train = train[["id"]].copy()
links_categorical_train = links_categorical_train.merge(links_train.reset_index(), how="left", on="id")
links_categorical_train[["links_cant","cant_failed_links"]] = links_categorical_train[["links_cant","cant_failed_links"]].fillna(0)

In [214]:
links_categorical_train = links_categorical_train[["id","links_cant", "links_encription_type", "cant_failed_links","links_domain_str"]]
links_categorical_train.head()

,id,links_cant,links_encription_type,cant_failed_links,links_domain_str
0,1,0.0,NaN,0.0,NaN
1,4,0.0,NaN,0.0,NaN
2,5,0.0,NaN,0.0,NaN
3,6,0.0,NaN,0.0,NaN
4,7,0.0,NaN,0.0,NaN


In [215]:
# test

links_categorical_test = test[["id"]].copy()
links_categorical_test = links_categorical_test.merge(links_test.reset_index(), how="left", on="id")
links_categorical_test[["links_cant","cant_failed_links"]] = links_categorical_test[["links_cant","cant_failed_links"]].fillna(0)

In [216]:
links_categorical_test = links_categorical_test[["id","links_cant", "links_encription_type", "cant_failed_links","links_domain_str"]]
links_categorical_test.head()

,id,links_cant,links_encription_type,cant_failed_links,links_domain_str
0,0,0.0,NaN,0.0,NaN
1,2,0.0,NaN,0.0,NaN
2,3,0.0,NaN,0.0,NaN
3,9,0.0,NaN,0.0,NaN
4,11,0.0,NaN,0.0,NaN


### Tags

In [297]:
tags_categorical_train = train_info[["id","cant_tags","tags_str"]]
tags_categorical_train.head()

,id,cant_tags,tags_str
0,1,2,"earthquake,reason"
1,4,2,"fire,forest"
2,5,2,"evacuation,officer"
3,6,4,"california,evacuation,people,wildfire"
4,7,3,"photo,smoke,wildfire"


In [298]:
tags_categorical_test =  test_info[["id","cant_tags","tags_str"]]
tags_categorical_test.head()

,id,cant_tags,tags_str
0,0,1,crash
1,2,3,"city,earthquake,heard"
2,3,3,"fire,forest,street"
3,9,2,"apocalypse,wildfire"
4,11,3,"china,kill,typhoon"


## Save csv categorical

In [299]:
keywords_categorical_train.to_csv('features/train/keywords_categorical_train.csv', encoding='utf-8')
keywords_categorical_test.to_csv('features/test/keywords_categorical_test.csv', encoding='utf-8')

text_categorical_features_train.to_csv('features/train/text_categorical_train.csv', encoding='utf-8')
text_categorical_features_test.to_csv('features/test/text_categorical_test.csv', encoding='utf-8')

location_categorical_train.to_csv('features/train/location_categorical_train.csv', encoding='utf-8')
location_categorical_test.to_csv('features/test/location_categorical_test.csv', encoding='utf-8')

links_categorical_train.to_csv('features/train/links_categorical_train.csv', encoding='utf-8')
links_categorical_test.to_csv('features/test/links_categorical_test.csv', encoding='utf-8')

tags_categorical_train.to_csv('features/train/tags_categorical_train.csv', encoding='utf-8')
tags_categorical_test.to_csv('features/test/tags_categorical_test.csv', encoding='utf-8')